<h3>Spark combineByKey assignment</h3>
In this assignment, you'll use combineByKey to find</li>
<ul>
    <li>the total number of taxi rides for each month in 2020</li>
    <li>the average total taxi fare for each month in 2020</li>
    <li>the standard deviation of the total fare for each month in 2020</li>
    <li>draw a simple graph using this data</li>
</ul>

The data file is available at <a href="https://drive.google.com/file/d/1EMKVVp33U6_F3YOKOlnagqZ4RQlY2fjY/view?usp=drive_link">https://drive.google.com/file/d/1EMKVVp33U6_F3YOKOlnagqZ4RQlY2fjY/view?usp=drive_link</a>

<h3>Packages</h3>
You're going to need several packages for the charting part of this assignment. 

<h3>Run this cell if you're using Apache Torre on GCP</h3>
<li>Note that this cell must be the first cell you run after loading the file or restarting the kernel</li>
<li>The packages take some time to load. Wait for all of them to load before proceeding</li>

In [ ]:
%AddDeps com.github.wookietreiber scala-chart_2.12 0.5.1
%AddDeps org.jfree jfreechart 1.0.19
%AddDeps org.scala-lang.modules scala-swing_2.12 3.0.0
%AddDeps jfree jcommon 1.0.0

<h3>Run this cell if you're using spylon kernel locally</h3>
<li>Note that this cell must be the first cell you run after loading the file or restarting the kernel</li>
<li>Note also that while you can use spylon kernel to test out your code, you must submit the GCP version of your assignment</li>
<li>Be warned. It will be hard to deal with the 11 GB file on a local machine</li>

In [ ]:
%%init_spark
launcher.num_executors = 4
launcher.executor_cores = 2
launcher.driver_memory = '4g'
launcher.conf.set("spark.sql.catalogImplementation", "hive")
launcher.packages = ["com.github.wookietreiber:scala-chart_2.12:0.5.1",
                    "org.jfree:jfreechart:1.0.19",
                    "org.scala-lang.modules:scala-swing_2.12:3.0.0",
                    "jfree:jcommon:1.0.0"]

<h3>Place your entire code in the next cell</h3>
<li>You can use the steps that follow to construct your code but <b>only the code in the next cell will be evaluated by the TAs!</b></li>
<li>Submission requirements</li>
<li><b>The notebook with the result (YYYYMM,(count, mean, stddev)) clearly visible in the output</b></li>
<li><b>The graph with your cluster information</b></li>

In [ ]:
// YOUR ENTIRE CODE SHOULD BE IN THIS CELL










<h4>Read the data and construct (key,value) paired data</h4>
<li>You need to give the full path to the file (gs://.....)</li>
<li>We are interested in the pickup date (column 2) and the total amount (column 18)</li>
<li>Since there are missing values (no total amount) and bad data (dates from 2018, 2003, etc.). Since we're only interested in 2017 data, your code should only return data from 2017. And, if there is any bad data, you need to get rid of it)</li>
<li>Each line should contain 20 values. Because the split function in Scala omits trailing comma sequences (see example below), you need to count the number of commas in the input line string. There should be exactly 19</li>
<li>write a function <span style="color:green">extract_data</span> that returns either Some (good data) or None (bad data). Use flatMap to get rid of the bad data</li>
<li>For example: </li>
<pre>
extractData("0,1,2018-01-01 00:32:05,2017-01-01 00:37:48,1,1.2,1,N,140,236,2,6.5,0.5,0.5,0.0,0.0,0.3,7.8,,")
</pre>
should return None (2018 is the wrong year)
<li>while:</li>
<pre>
extractData("0,1,2017-01-01 00:32:05,2017-01-01 00:37:48,1,1.2,1,N,140,236,2,6.5,0.5,0.5,0.0,0.0,0.3,7.8,,")
</pre>
should return Some((201701,7.8))


In [ ]:
//Scala skips trailing comma sequences
"1,1,,".split(',') //should return Array("1","1","") but returns Array("1","1")

//The Scala count method takes a conditional boolean as an argument
"John Joe Bodega".count(_ == 'o') //returns 3 (the count of the letter o)

In [ ]:
def extractData(line: String):Option[(String,Double)] = {
    try {
        if (line.count(_ ==',') != 19) None
        else {
            val array = line.split(",")
            val tup = (array(2),array(17).toDouble)
            val yyyy = tup._1.slice(0,4)
            if (yyyy != "2017")
                None
            else {
                val yyyymm = tup._1.slice(0,4) ++ tup._1.slice(5,7)
                val final_data = (tup._1.slice(0,4) ++ tup._1.slice(5,7),tup._2)
                Some(final_data)
            }
        }
    } catch {
        case e: Exception => None
    }
}
extractData("0,1,2017-01-01 00:32:05,2017-01-01 00:37:48,1,1.2,1,N,140,236,2,6.5,0.5,0.5,0.0,0.0,0.3,7.8,,")

<h3>Preapare the key,value pairs</h3>
<li>read the data</li>
<li>construct key value pairs using extractData</li>
<li>data_rdd size: 113,500,159</li>

In [ ]:
//GCP bucket
//val url = "gs://hj2203-fall-2023/data/taxi_data_2017.csv"
//Local file
val url = "/Users/hardeepjohar/Downloads/tmp/taxi_data_2017.csv"

val data_rdd = sc.textFile(url)
    .flatMap(r => extractData(r))


<h3>Write the combiner, merger, mergeAndCombiner and getAvgAndStd functions</h3>
<li>Think about what goes into each and how you should accumulate values</li>
<li>For standard deviation, use the formula below (though accuracy is not guaranteed!)</li>
<li>This makes it possible to calculate mean and square root with only one pass through the data</li>
<p>
    </p>
$ \sqrt{\frac{1}{N}\Sigma x_{i}^{2}-\mu^{2}} $

In [ ]:
val combiner = (x: Double) => (1,x,x*x)
val merger = (x: (Int, Double, Double),y: Double) => {
    val (c,acc_sum,acc_squares) = x
    (c+1, acc_sum + y, acc_squares + y*y)
}
val mergeAndCombiner = (x1: (Int, Double, Double), x2: (Int, Double, Double)) => {
    val (c1,acc_sum1,acc_squares1) = x1
    val (c2,acc_sum2,acc_squares2) = x2
    (c1+c2,acc_sum1+acc_sum2,acc_squares1 + acc_squares2)
}

val getAvgAndStd = (x: (String, (Int, Double, Double))) => {
    val (identifier, (count,acc_sum,acc_squares)) = x
    val mean = acc_sum/count.toDouble
    val stdev = math.sqrt(1.0/count.toDouble*acc_squares - acc_sum*acc_sum/(count.toDouble*count.toDouble))
    (identifier,(count,mean,stdev))
}



In [ ]:
val result = data_rdd.combineByKey(combiner,merger,mergeAndCombiner).map(getAvgAndStd)

<h3>Get the result in a scala Array</h3>
<li>There should be 12 lines of data, 201701, 201702, ...,201712</li>
<li>Sort the data in order of month (see https://alvinalexander.com/scala/how-to-sort-map-in-scala-key-value-sortby-sortwith/)</li>

In [ ]:
val scala_result = result.collect.sortBy(_._1)

In [ ]:
scala_result.foreach(println)

<h3>Draw a bar chart comparing number of monthly rides</h3>
<li>Save it to a file (use the location /tmp/rides.png)</li>
<li>Do necessary imports and see the example below</li>
<li>To access the saved file, in the Jupyter file navigator, go to /Local Disk/tmp and look for this file. Note that the file will disappear when you delete the cluster!</li>
<li>Open the file in your browser and screenshot it. Make sure that your cluster information is included in the screenshot</li>

In [ ]:
import scalax.chart.api._
import org.jfree._

In [ ]:
//data is a Scala Vector object. To convert an Array to a Vector, use .toVector
val data = for (i <- 1 to 5) yield (i,i*i)
val chart = XYBarChart(data)
//val chart = XYBarChart(data)

In [ ]:
val data = scala_result.map(t => (t._1.toInt,t._2._1)).toVector
val chart = XYBarChart(data)
chart.saveAsPNG("sample_chart.png")